In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import datetime
import numpy as np
import os
import pandas as pd

In [2]:
# Suppress TensorFlow logging
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

print(f"Using TensorFlow version: {tf.__version__}\n")

Using TensorFlow version: 2.19.0



## 1. Environment Setup and Data Loading

### 1.1. Load the Fashion-MNIST dataset

In [3]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()

29515/29515 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
26421880/26421880 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
5148/5148 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
4422102/4422102 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


### 1.2. Normalize the image data to [0, 1]

In [4]:
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

### 1.3. Reshape data for the CNN (N, 28, 28, 1)

We add the 'channels' dimension

In [5]:
x_train_cnn = np.expand_dims(x_train, -1)
x_test_cnn = np.expand_dims(x_test, -1)

In [6]:
print(f"Original x_train shape: {x_train.shape}")
print(f"Data shape for MLP (no change needed, Flatten layer handles): {x_train.shape}")
print(f"Data shape for CNN (with channels dim): {x_train_cnn.shape}\n")

Original x_train shape: (60000, 28, 28)
Data shape for MLP (no change needed, Flatten layer handles): (60000, 28, 28)
Data shape for CNN (with channels dim): (60000, 28, 28, 1)



## 2. Model Implementation

### 2.1 MLP model
#### 2.1.1 MLP model implementaion

In [7]:
mlp_model = keras.Sequential([
    layers.Flatten(input_shape=(28, 28)),
    layers.Dense(256, activation='relu'),
    layers.Dense(128, activation='relu'),
    layers.Dense(10, activation='softmax')
], name="MLP_Model")

/usr/local/lib/python3.12/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


### 2.1.2 Compile the MLP model

In [8]:
mlp_model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

### 2.1.3 MLP model summary

In [9]:
mlp_model.summary()

Model: "MLP_Model"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ flatten (Flatten)               │ (None, 784)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │       200,960 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 10)             │         1,290 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 235,146 (918.54 KB)

 Trainable params: 235,146 (918.54 KB)

 Non-trainable params: 0 (0.00 B)

### 2.2 CNN model
#### 2.2.1 CNN model implementaion

In [10]:
cnn_model = keras.Sequential([
    # Convolutional Block 1
    layers.Conv2D(16, kernel_size=3, activation='relu', input_shape=(28, 28, 1)),
    layers.MaxPooling2D(pool_size=2),

    # Convolutional Block 2
    layers.Conv2D(32, kernel_size=3, activation='relu'),
    layers.MaxPooling2D(pool_size=2),

    # Classifier
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(10, activation='softmax')
], name="CNN_Model")

/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


#### 2.2.2 Compile the CNN model

In [11]:
cnn_model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy'],
)

#### 2.2.3 The CNN model summary

In [12]:
cnn_model.summary()

Model: "CNN_Model"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 26, 26, 16)     │           160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 13, 13, 16)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 11, 11, 32)     │         4,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 5, 5, 32)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 800)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 64)             │        51,264 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 10)             │           650 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 56,714 (221.54 KB)

 Trainable params: 56,714 (221.54 KB)

 Non-trainable params: 0 (0.00 B)

###

## 3. Training and Evaluation

### 3.1 Train MLP model

In [13]:
mlp_history = mlp_model.fit(
    x_train, y_train,
    epochs=5,
    batch_size=64,
    verbose=1
)

Epoch 1/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 6s 5ms/step - accuracy: 0.7817 - loss: 0.6356
Epoch 2/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 6s 6ms/step - accuracy: 0.8674 - loss: 0.3630
Epoch 3/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 6s 6ms/step - accuracy: 0.8806 - loss: 0.3206
Epoch 4/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 0.8898 - loss: 0.2995
Epoch 5/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 6s 6ms/step - accuracy: 0.8948 - loss: 0.2866


### 3.1 Train CNN model

In [14]:
cnn_history = cnn_model.fit(
    x_train_cnn, y_train,
    epochs=5,
    batch_size=64,
    verbose=1
)

Epoch 1/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 25s 26ms/step - accuracy: 0.7171 - loss: 0.7957
Epoch 2/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 28s 29ms/step - accuracy: 0.8620 - loss: 0.3837
Epoch 3/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 29s 31ms/step - accuracy: 0.8798 - loss: 0.3337
Epoch 4/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 26s 27ms/step - accuracy: 0.8894 - loss: 0.3053
Epoch 5/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 24s 26ms/step - accuracy: 0.8982 - loss: 0.2794


### 3.3. Evaluate and Report

#### 3.3.1 MLP model

In [15]:
mlp_loss, mlp_acc = mlp_model.evaluate(x_test, y_test, verbose=0)
print(f"MLP Model - Test Loss:     {mlp_loss:.4f}")
print(f"MLP Model - Test Accuracy: {mlp_acc * 100:.2f}%")

MLP Model - Test Loss:     0.3526
MLP Model - Test Accuracy: 87.73%


#### 3.3.2 CNN model

In [16]:
cnn_loss, cnn_acc = cnn_model.evaluate(x_test_cnn, y_test, verbose=0)
print(f"CNN Model - Test Loss:     {cnn_loss:.4f}")
print(f"CNN Model - Test Accuracy: {cnn_acc * 100:.2f}%")

CNN Model - Test Loss:     0.3152
CNN Model - Test Accuracy: 88.42%


## 4. Resource Usage Comparison

In [17]:
mlp_params = mlp_model.count_params()
cnn_params = cnn_model.count_params()

print(f"MLP Trainable Parameters: {mlp_params}")
print(f"CNN Trainable Parameters: {cnn_params}")

MLP Trainable Parameters: 235146
CNN Trainable Parameters: 56714


### 4.1 Save the models

In [18]:
mlp_model.save('mlp_model.keras')
cnn_model.save('cnn_model.keras')

### 4.2 Get file sizes in MB

In [20]:
mlp_size_mb = os.path.getsize('mlp_model.keras') / (1024 * 1024)
cnn_size_mb = os.path.getsize('cnn_model.keras') / (1024 * 1024)

print(f"MLP Saved Model Size: {mlp_size_mb:.2f} MB")
print(f"CNN Saved Model Size: {cnn_size_mb:.2f} MB")

MLP Saved Model Size: 2.72 MB
CNN Saved Model Size: 0.68 MB


### 4.3. Estimate Computational Resources (FLOPs & Training Memory)

Calculating exact FLOPs and memory is complex.
we gonna use other library to calucalte the FLOPS

#### 4.3.1. FLOPs (Floating-point operations)

In [21]:
def safe_count_flops(model, input_shape):
    """
    Counts FLOPs safely across TensorFlow versions.
    Uses built-in Model.count_flops() if available (TF ≥ 2.16),
    otherwise falls back to a profiler-based method (TF ≤ 2.15).
    """
    # --- Modern API path (TensorFlow 2.16+ / Keras 3.x) ---
    if hasattr(model, "count_flops"):
        flops = model.count_flops(
            input_signature=(tf.TensorSpec(shape=(1, *input_shape), dtype=tf.float32),)
        )
        return flops

    # --- Backward-compatible path ---
    from tensorflow.python.profiler import model_analyzer, option_builder
    from tensorflow.python.framework import convert_to_constants

    concrete_func = tf.function(model).get_concrete_function(
        tf.TensorSpec([1, *input_shape], tf.float32)
    )
    frozen_func = convert_to_constants.convert_variables_to_constants_v2(concrete_func)

    run_meta = tf.compat.v1.RunMetadata()
    opts = option_builder.ProfileOptionBuilder.float_operation()

    flops = tf.compat.v1.profiler.profile(
        graph=frozen_func.graph,
        run_meta=run_meta,
        cmd='op',
        options=opts
    )
    return flops.total_float_ops


# --- Use it on your models ---
mlp_flops = safe_count_flops(mlp_model, input_shape=(28, 28))
cnn_flops = safe_count_flops(cnn_model, input_shape=(28, 28, 1))

print(f"MLP Model FLOPs: {mlp_flops / 1e6:.2f} M-FLOPs")
print(f"CNN Model FLOPs: {cnn_flops / 1e6:.2f} M-FLOPs")

Instructions for updating:
This API was designed for TensorFlow v1. See https://www.tensorflow.org/guide/migrate for instructions on how to migrate your code to TensorFlow v2.


MLP Model FLOPs: 0.47 M-FLOPs
CNN Model FLOPs: 1.44 M-FLOPs


#### 4.3.2. Training Memory

Training memory ≈ (Model Parameters + Gradients + Optimizer State) + Activations
- Model Parameters: Size of weights (e.g., mlp_params \* 4 bytes for float32)
- Gradients: Same size as parameters.
- Optimizer State: For Adam, stores 2 states (m, v) per param. (2 \* param_size)
- Activations: Depends on batch_size. (batch_size \* activation_size_per_layer)

Rough Estimate (Model-related memory, excluding activations):
Total = Params + Gradients + Optimizer(m) + Optimizer(v) = 4 \* Param_size

In [22]:
mlp_train_mem_mb = (mlp_params * 4 * 4) / (1024 * 1024) # 4x params, 4 bytes/param
cnn_train_mem_mb = (cnn_params * 4 * 4) / (1024 * 1024) # 4x params, 4 bytes/param
print(f"Estimated MLP Training Memory (Params + Grads + Adam): {mlp_train_mem_mb:.2f} MB")
print(f"Estimated CNN Training Memory (Params + Grads + Adam): {cnn_train_mem_mb:.2f} MB")
print("(This excludes memory for batch activations, which can be significant)")

Estimated MLP Training Memory (Params + Grads + Adam): 3.59 MB
Estimated CNN Training Memory (Params + Grads + Adam): 0.87 MB
(This excludes memory for batch activations, which can be significant)


## 5. Final Report and Conclusion

### 5.1. Write the Conclusion

create a model summary

In [23]:
summary_data = {
    "Model": ["MLP", "CNN"],
    "Test Accuracy": [f"{mlp_acc*100:.2f}%", f"{cnn_acc*100:.2f}%"],
    "Trainable Parameters": [mlp_params, cnn_params],
    "Saved Model Size (MB)": [f"{mlp_size_mb:.2f}", f"{cnn_size_mb:.2f}"],
    "FLOPs (Inference)": ["0.47 M-FLOPs", "1.44 M-FLOPs"],
    "FLOPs (Training)": ["~1.4 MFLOPs (Est.)", "~4.24 MFLOPs (Est.)"],
    "Training Memory (MB)": [f"~{mlp_train_mem_mb:.2f} + Activations", f"~{cnn_train_mem_mb:.2f} + Activations"]
}

summary_table = pd.DataFrame(summary_data)
print(summary_table.to_markdown(index=False, numalign="center", stralign="center"))

|  Model  |  Test Accuracy  |  Trainable Parameters  |  Saved Model Size (MB)  |  FLOPs (Inference)  |  FLOPs (Training)   |  Training Memory (MB)  |
|:-------:|:---------------:|:----------------------:|:-----------------------:|:-------------------:|:-------------------:|:----------------------:|
|   MLP   |     87.73%      |         235146         |          2.72           |    0.47 M-FLOPs     | ~1.4 MFLOPs (Est.)  |  ~3.59 + Activations   |
|   CNN   |     88.42%      |         56714          |          0.68           |    1.44 M-FLOPs     | ~4.24 MFLOPs (Est.) |  ~0.87 + Activations   |


|  Model  |  Test Accuracy  |  Trainable Parameters  |  Saved Model Size (MB)  |  FLOPs (Inference)  |  FLOPs (Training)  |  Training Memory (MB)  |
|:-------:|:---------------:|:----------------------:|:-----------------------:|:-------------------:|:------------------:|:----------------------:|
|   MLP   |     88.17%      |         235146         |          2.72           | ~470 kFLOPs (Est.)  | ~1.4 MFLOPs (Est.) |  ~3.59 + Activations   |
|   CNN   |     88.98%      |         56714          |          0.69           | ~1.2 MFLOPs (Est.)  | ~3.6 MFLOPs (Est.) |  ~0.87 + Activations   |

## Conclusion Questions

### 1. Which model achieved a higher accuracy?

Answer: The **CNN model** achieved higher accuracy (88.98%) compared to the MLP model (88.17%).

### 2. Which model had a smaller number of parameters (lower memory footprint)?
Answer: The **CNN model** had significantly fewer parameters (56714) than the MLP model (235146). This also resulted in a smaller saved file size.


### 3. Explain the trade-off between the two models...

The **CNN model** (the "winner" in accuracy) is superior for image tasks because it uses **convolutional layers**. These layers are specifically designed to find spatial patterns (like edges, textures, and shapes) in the image. They also use **parameter sharing**, where the same filter (kernel) is slid across the entire image. This makes the model:
   1.  **More effective:** It learns features that are *translationally invariant* (it can find a "shoe" anywhere in the image).
   2.  **More efficient:** It needs far fewer parameters than an MLP, which connects *every* input pixel to *every* neuron in the first hidden layer.


The **MLP model**'s main disadvantage is that it's "fully connected." It treats the $28 \times 28$ image as a flat $784$-element vector and loses all spatial information. It doesn't know which pixels are next to each other. Its only "advantage" in this context is its conceptual simplicity, but it is a poor choice for image data, as shown by its lower accuracy and massive parameter count.


**In summary:** The CNN achieves higher accuracy with *fewer* parameters because it is an architecture fundamentally suited for spatial data like images.

## 6. Lab 6

### 6.1 Loading keras models

In [24]:
mlp_model = tf.keras.models.load_model('mlp_model.keras')
cnn_model = tf.keras.models.load_model('cnn_model.keras')

### 6.2. Load and Prepare the Representative Dataset
This is required for Full Integer Quantization
We only need a small sample of the training data for calibration.

In [25]:
(x_train, _), (_, _) = tf.keras.datasets.fashion_mnist.load_data()
x_train = x_train.astype('float32') / 255.0

In [26]:
# Representative data for MLP (input shape 28, 28)
rep_data_mlp = x_train[:300] # 300 samples is plenty

In [27]:
# Representative data for CNN (input shape 28, 28, 1)
x_train_cnn = np.expand_dims(x_train, -1)
rep_data_cnn = x_train_cnn[:300] # 300 samples

In [28]:
print(f"Using {len(rep_data_mlp)} samples for calibration.")

Using 300 samples for calibration.


### 6.3. Define the Representative Dataset Generator

In [29]:
def representative_dataset_gen(data):
    def gen():
        for image in data:
            # Yield the image, cast to float32, and add a batch dimension
            yield [np.expand_dims(image, axis=0).astype(np.float32)]
    return gen

### 6.4. Task 3.1: Convert and Quantize MLP Model

In [30]:
# Create the generator for the MLP data
mlp_rep_gen = representative_dataset_gen(rep_data_mlp)

converter_mlp = tf.lite.TFLiteConverter.from_keras_model(mlp_model)
converter_mlp.optimizations = [tf.lite.Optimize.DEFAULT]
converter_mlp.representative_dataset = mlp_rep_gen
# Enforce full integer quantization for microcontroller compatibility
converter_mlp.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter_mlp.inference_input_type = tf.uint8  # Input will be 0-255 images
converter_mlp.inference_output_type = tf.uint8 # Output will be quantized scores

tflite_model_quant_mlp = converter_mlp.convert()

# Save the quantized model
with open('mlp_model_quant_int8.tflite', 'wb') as f:
    f.write(tflite_model_quant_mlp)

print("MLP TFLite model converted and saved as 'mlp_model_quant_int8.tflite'")

Saved artifact at '/tmp/tmpylbg00xa'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 28, 28), dtype=tf.float32, name='input_layer')
Output Type:
  TensorSpec(shape=(None, 10), dtype=tf.float32, name=None)
Captures:
  132545097605264: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132545097613136: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132545097598544: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132545097612176: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132545097601040: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132545097605648: TensorSpec(shape=(), dtype=tf.resource, name=None)


/usr/local/lib/python3.12/dist-packages/tensorflow/lite/python/convert.py:854: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn(


MLP TFLite model converted and saved as 'mlp_model_quant_int8.tflite'


### 5.5. Task 3.2: Convert and Quantize CNN Model

In [31]:
# Create the generator for the CNN data
cnn_rep_gen = representative_dataset_gen(rep_data_cnn)

converter_cnn = tf.lite.TFLiteConverter.from_keras_model(cnn_model)
converter_cnn.optimizations = [tf.lite.Optimize.DEFAULT]
converter_cnn.representative_dataset = cnn_rep_gen
# Enforce full integer quantization
converter_cnn.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter_cnn.inference_input_type = tf.uint8
converter_cnn.inference_output_type = tf.uint8

tflite_model_quant_cnn = converter_cnn.convert()

# Save the quantized model
with open('cnn_model_quant_int8.tflite', 'wb') as f:
    f.write(tflite_model_quant_cnn)

print("CNN TFLite model converted and saved as 'cnn_model_quant_int8.tflite'")

Saved artifact at '/tmp/tmpt1z1g43s'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 28, 28, 1), dtype=tf.float32, name='input_layer_1')
Output Type:
  TensorSpec(shape=(None, 10), dtype=tf.float32, name=None)
Captures:
  132545086198928: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132545086199696: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132545086202960: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132545086202000: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132545086199888: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132545086203152: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132545086202576: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132545086203536: TensorSpec(shape=(), dtype=tf.resource, name=None)


/usr/local/lib/python3.12/dist-packages/tensorflow/lite/python/convert.py:854: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn(


CNN TFLite model converted and saved as 'cnn_model_quant_int8.tflite'


### 6.6. Task 4.1: Report File Sizes

In [32]:
# Get original Keras model sizes
mlp_keras_size_kb = os.path.getsize('mlp_model.keras') / 1024
cnn_keras_size_kb = os.path.getsize('cnn_model.keras') / 1024

In [33]:
# Get new TFLite model sizes
mlp_tflite_size_kb = os.path.getsize('mlp_model_quant_int8.tflite') / 1024
cnn_tflite_size_kb = os.path.getsize('cnn_model_quant_int8.tflite') / 1024

In [34]:
print("\nOriginal Keras Models (float32):")
print(f"  MLP Keras Size: {mlp_keras_size_kb:.2f} KB ({mlp_keras_size_kb/1024:.2f} MB)")
print(f"  CNN Keras Size: {cnn_keras_size_kb:.2f} KB ({cnn_keras_size_kb/1024:.2f} MB)")


Original Keras Models (float32):
  MLP Keras Size: 2782.62 KB (2.72 MB)
  CNN Keras Size: 701.13 KB (0.68 MB)


In [35]:
print("\nQuantized TFLite Models (int8):")
print(f"  MLP TFLite Size:  {mlp_tflite_size_kb:.2f} KB")
print(f"  CNN TFLite Size:  {cnn_tflite_size_kb:.2f} KB")

print("\nConstraint:")
print(f"  XIAO ESP32S3 SRAM: 512 KB")


Quantized TFLite Models (int8):
  MLP TFLite Size:  243.04 KB
  CNN TFLite Size:  62.77 KB

Constraint:
  XIAO ESP32S3 SRAM: 512 KB


| Model | Keras Size (Float32, MB) | Quantized TFLite Size (int8, MB) | SRAM Constraint (XIAO) | Can Model Fit in SRAM? |
|--------|---------------------------|----------------------------------|------------------------|------------------------|
| MLP    | ≈0.9                     | ≈0.24 MB (243.04 KB)            | 512 KB (0.5 MB)        | Yes                    |
| CNN    | ≈0.2                     | ≈0.06 MB (62.77 KB)             | 512 KB (0.5 MB)        | Yes                    |
